This notebook contains the code and analysis for analyzing the SciPlex dataset for coding assignment 1 in CS 582.

In [ ]:
# Python standard library imports
import os

# Third-party libraries
import numpy as np
import pandas as pd

# Libraries for single-cell analysis
import anndata
import scanpy